In [89]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

### Loading all datasets - genuine and bot accounts

In [90]:
genuine_users = pd.read_csv("genuine_accounts_users.csv")
genuine_users.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,...,notifications,description,contributors_enabled,following,created_at,timestamp,crawled_at,updated,test_set_1,test_set_2
0,1502026416,TASUKU HAYAKAWA,0918Bask,2177,208,332,265,1,NaN,ja,...,NaN,15years ago X.Lines24,NaN,NaN,Tue Jun 11 11:20:35 +0000 2013,2013-06-11 13:20:35,2015-05-02 06:41:46,2016-03-15 15:53:47,0,0
1,2492782375,ro_or,1120Roll,2660,330,485,3972,5,NaN,ja,...,NaN,保守見習い地元大好き人間。 経済学、電工、仏教を勉強中、ちなDeではいかんのか？ (*^◯^*),NaN,NaN,Tue May 13 10:37:57 +0000 2014,2014-05-13 12:37:57,2015-05-01 17:20:27,2016-03-15 15:53:48,0,0
2,293212315,bearclaw,14KBBrown,1254,166,177,1185,0,NaN,en,...,NaN,Let me see what your best move is!,NaN,NaN,Wed May 04 23:30:37 +0000 2011,2011-05-05 01:30:37,2015-05-01 18:48:28,2016-03-15 15:53:48,0,0
3,191839658,pocahontas farida,wadespeters,202968,2248,981,60304,101,http://t.co/rGV0HIJGsu,en,...,NaN,20. menna: #farida #nyc and the 80s actually y...,NaN,NaN,Fri Sep 17 14:02:10 +0000 2010,2010-09-17 16:02:10,2015-05-01 13:55:16,2016-03-15 15:53:48,0,0
4,3020965143,Ms Kathy,191a5bd05da04dc,82,21,79,5,0,NaN,en,...,NaN,Cosmetologist,NaN,NaN,Fri Feb 06 04:10:49 +0000 2015,2015-02-06 05:10:49,2015-05-02 01:17:32,2016-03-15 15:53:48,0,0


In [91]:
genuine_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3474 entries, 0 to 3473
Data columns (total 42 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  3474 non-null   int64  
 1   name                                3473 non-null   object 
 2   screen_name                         3474 non-null   object 
 3   statuses_count                      3474 non-null   int64  
 4   followers_count                     3474 non-null   int64  
 5   friends_count                       3474 non-null   int64  
 6   favourites_count                    3474 non-null   int64  
 7   listed_count                        3474 non-null   int64  
 8   url                                 1266 non-null   object 
 9   lang                                3474 non-null   object 
 10  time_zone                           2475 non-null   object 
 11  location                            2365 no

In [92]:
social_spambots_1 = pd.read_csv("social_spambots_1_users.csv")
social_spambots_1.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,...,verified,notifications,description,contributors_enabled,following,created_at,timestamp,crawled_at,updated,test_set_1
0,24858289,Davide Bertoli,davideb66,1299,22,40,1,0,NaN,it,...,NaN,NaN,NaN,NaN,NaN,Tue Mar 17 08:51:12 +0000 2009,2009-03-17 09:51:12,2014-04-19 14:46:19,2016-03-15 14:12:22,1
1,33212890,Elisa D'Ospina,ElisaDospina,18665,12561,3442,16358,110,http://t.co/ceK8TovxwI,it,...,NaN,NaN,Autrice del libro #unavitatuttacurve dal 9 apr...,NaN,NaN,Sun Apr 19 14:38:04 +0000 2009,2009-04-19 16:38:04,2014-05-18 23:20:58,2016-03-15 14:17:13,1
2,39773427,Donato Vincenzo,Vladimir65,22987,600,755,14,6,NaN,it,...,NaN,NaN,[Live Long and Prosper],NaN,NaN,Wed May 13 15:34:41 +0000 2009,2009-05-13 17:34:41,2014-05-13 23:21:54,2016-03-15 14:16:44,1
3,57007623,Rafiela Morales L.,RafielaMorales,7975,398,350,11,2,NaN,en,...,NaN,NaN,"Cuasi Odontologa*♥,#Bipolar, #Sarcastica & Som...",NaN,NaN,Wed Jul 15 12:55:03 +0000 2009,2009-07-15 14:55:03,2014-05-19 23:24:18,2016-03-15 14:18:54,1
4,63258466,§ h a † u r♄,FabrizioC_c,20218,413,405,162,8,http://t.co/PK5F0JDKcy,it,...,NaN,NaN,"I shall rise from my own death, to avenge hers...",NaN,NaN,Wed Aug 05 21:12:49 +0000 2009,2009-08-05 23:12:49,2014-05-11 23:22:23,2016-03-15 14:17:05,1


In [93]:
social_spambots_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991 entries, 0 to 990
Data columns (total 41 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  991 non-null    int64  
 1   name                                991 non-null    object 
 2   screen_name                         991 non-null    object 
 3   statuses_count                      991 non-null    int64  
 4   followers_count                     991 non-null    int64  
 5   friends_count                       991 non-null    int64  
 6   favourites_count                    991 non-null    int64  
 7   listed_count                        991 non-null    int64  
 8   url                                 22 non-null     object 
 9   lang                                991 non-null    object 
 10  time_zone                           903 non-null    object 
 11  location                            745 non-n

In [94]:
social_spambots_2 = pd.read_csv("social_spambots_2_users.csv")
social_spambots_2.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,...,protected,verified,notifications,description,contributors_enabled,following,created_at,timestamp,crawled_at,updated
0,2372241176,Denna Mcsparren,DennaMcsparren,53,10,46,0,0,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,Tue Mar 04 18:11:08 +0000 2014,2014-03-04 19:11:08,2014-05-05 00:20:03,2016-03-15 15:02:07
1,2368684734,Yukiko Tretter,YukikoTretter,68,4,40,0,0,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,Sun Mar 02 10:38:13 +0000 2014,2014-03-02 11:38:13,2014-05-05 00:20:47,2016-03-15 15:02:07
2,2353855646,Rochel Amaro,RochelAmaro,79,9,39,0,0,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,Thu Feb 20 22:28:03 +0000 2014,2014-02-20 23:28:03,2014-05-05 00:20:03,2016-03-15 15:02:08
3,2372322542,Brandi Babin,BrandiBabin,59,1,39,0,0,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,Tue Mar 04 19:52:10 +0000 2014,2014-03-04 20:52:10,2014-05-05 00:20:03,2016-03-15 15:02:08
4,2352506778,Chung Posadas,ChungPosadas,73,7,36,0,0,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,Thu Feb 20 01:34:19 +0000 2014,2014-02-20 02:34:19,2014-05-05 00:20:03,2016-03-15 15:02:09


In [95]:
social_spambots_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3457 entries, 0 to 3456
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  3457 non-null   int64  
 1   name                                3457 non-null   object 
 2   screen_name                         3457 non-null   object 
 3   statuses_count                      3457 non-null   int64  
 4   followers_count                     3457 non-null   int64  
 5   friends_count                       3457 non-null   int64  
 6   favourites_count                    3457 non-null   int64  
 7   listed_count                        3457 non-null   int64  
 8   url                                 10 non-null     object 
 9   lang                                3457 non-null   object 
 10  time_zone                           14 non-null     object 
 11  location                            13 non-

In [96]:
social_spambots_3 = pd.read_csv("social_spambots_3_users.csv")
social_spambots_3.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,...,verified,notifications,description,contributors_enabled,following,created_at,timestamp,crawled_at,updated,test_set_2
0,16282004,Enrique Kates,eckates,11405,819,2000,0,19,http://t.co/gAa6cVM0Fe,en,...,NaN,NaN,My name is Enrique! ! I'm a highly experienced...,NaN,NaN,Sun Sep 14 11:20:09 +0000 2008,2008-09-14 13:20:09,2014-05-05 23:17:51,2016-03-15 15:41:18,1
1,16740486,genebailey,genebailey,520,219,406,36,2,http://t.co/mag9oYulVZ,en,...,NaN,NaN,"Author, Speaker, Father, Friend, Motivator, Re...",NaN,NaN,Tue Oct 14 16:11:24 +0000 2008,2008-10-14 18:11:24,2014-05-05 23:17:51,2016-03-15 15:41:15,1
2,17132768,Patrick G Howard,patrickghoward,4671,38877,25953,6,228,http://t.co/0ukMNj4N3Y,en,...,NaN,NaN,Patrick G Howard is an experienced project & p...,NaN,NaN,Mon Nov 03 15:51:00 +0000 2008,2008-11-03 16:51:00,2014-05-05 23:17:51,2016-03-15 15:41:15,1
3,18013384,Doggie Cakes Bakery,DoggieCakes,8512,2069,1177,70,43,http://t.co/B4NRMJHH6Q,en,...,NaN,NaN,Dog Bakery and Boutique - Retail (Events and O...,NaN,NaN,Wed Dec 10 05:44:57 +0000 2008,2008-12-10 06:44:57,2014-05-05 23:12:36,2016-03-15 15:41:04,1
4,21331733,David Varrone,DavidVarrone,245,6656,7469,1,72,http://t.co/KLzAJ1yzmB,en,...,NaN,NaN,Home Based Business and Personal Development C...,NaN,NaN,Thu Feb 19 19:46:02 +0000 2009,2009-02-19 20:46:02,2014-05-05 23:17:51,2016-03-15 15:41:10,1


In [97]:
social_spambots_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464 entries, 0 to 463
Data columns (total 41 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  464 non-null    int64  
 1   name                                464 non-null    object 
 2   screen_name                         464 non-null    object 
 3   statuses_count                      464 non-null    int64  
 4   followers_count                     464 non-null    int64  
 5   friends_count                       464 non-null    int64  
 6   favourites_count                    464 non-null    int64  
 7   listed_count                        464 non-null    int64  
 8   url                                 253 non-null    object 
 9   lang                                464 non-null    object 
 10  time_zone                           303 non-null    object 
 11  location                            54 non-nu

In [98]:
traditional_spambots_1 = pd.read_csv("traditional_spambots_1_users.csv")
traditional_spambots_1.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,...,protected,verified,notifications,description,contributors_enabled,following,created_at,timestamp,crawled_at,updated
0,7248952,Bhuvan Chand,tarunkjuyal,1259,837,1978,3200,9,http://lifeofearth.org,NaN,...,NaN,NaN,NaN,Love Your Life,NaN,NaN,1183552203000L,2007-07-04 14:30:03,2010-11-07 11:10:52,2016-03-14 17:05:53
1,7732472,Daniel Wagner,DanielWagner,770,3274,3595,8,22,http://www.yourinternetbuddies.com/go,NaN,...,NaN,NaN,NaN,I am an internet marketing coach and mentor wh...,NaN,NaN,1185440851000L,2007-07-26 11:07:31,2010-11-07 11:10:52,2016-03-14 17:05:54
2,9524952,Andrew Lock,Andrewlock,1100,38849,34504,41,1014,http://www.helpmybusiness.com,NaN,...,NaN,NaN,NaN,Marketing Geek & Presenter of 'Help! My Busine...,NaN,NaN,1192725360000L,2007-10-18 18:36:00,2010-11-07 11:10:52,2016-03-14 17:05:54
3,10788822,Tim Thompson,yourinsaneworld,6497,5902,5496,0,82,http://investing-information.com,NaN,...,NaN,NaN,NaN,I am a member of a network of stock investing ...,NaN,NaN,1196614406000L,2007-12-02 17:53:26,2010-11-07 11:10:52,2016-03-14 17:05:54
4,14596967,fxgenie,fxgenie,3203,2570,2638,0,5,http://www.4xgenie.com/wp,NaN,...,NaN,NaN,NaN,forex trader,NaN,NaN,1209536534000L,2008-04-30 08:22:14,2010-11-07 11:10:52,2016-03-14 17:05:54


In [99]:
traditional_spambots_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  1000 non-null   int64  
 1   name                                1000 non-null   object 
 2   screen_name                         1000 non-null   object 
 3   statuses_count                      1000 non-null   int64  
 4   followers_count                     1000 non-null   int64  
 5   friends_count                       1000 non-null   int64  
 6   favourites_count                    1000 non-null   int64  
 7   listed_count                        1000 non-null   int64  
 8   url                                 396 non-null    object 
 9   lang                                0 non-null      float64
 10  time_zone                           0 non-null      float64
 11  location                            243 non-

In [100]:
traditional_spambots_2 = pd.read_csv("traditional_spambots_2_users.csv")
traditional_spambots_2.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,...,protected,verified,notifications,description,contributors_enabled,following,created_at,timestamp,crawled_at,updated
0,2355955040,Prize Giveaways,PrizeCrazy66450,716,86,78,0,4,http://t.co/CF0tmHKIHk,en-gb,...,NaN,NaN,NaN,NOTICE TO WINNERS: A unique prize code is nece...,NaN,NaN,Sat Feb 22 07:22:34 +0000 2014,2014-02-22 08:22:34,2014-05-05 21:59:19,2016-03-14 15:46:02
1,2368624550,Prize Giveaways,Prizetopia43484,779,87,79,0,2,http://t.co/3agfNy4i4S,en-gb,...,NaN,NaN,NaN,NOTICE TO WINNERS: A unique invite code is nee...,NaN,NaN,Sun Mar 02 09:38:19 +0000 2014,2014-03-02 10:38:19,2014-05-05 21:59:19,2016-03-14 15:46:02
2,2355950858,Prize Crazy,TweetWin57918,753,89,97,0,0,http://t.co/jlIaA4LovD,en-gb,...,NaN,NaN,NaN,NOTICE TO WINNERS: A unique invitation code is...,NaN,NaN,Sat Feb 22 07:19:00 +0000 2014,2014-02-22 08:19:00,2014-05-05 21:59:19,2016-03-14 15:46:02
3,2357744766,Prize World,Prizetopia67432,743,93,104,0,4,http://t.co/lVWwVDdjr6,en-gb,...,NaN,NaN,NaN,NOTICE TO WINNERS: A unique invitation code is...,NaN,NaN,Sun Feb 23 09:30:43 +0000 2014,2014-02-23 10:30:43,2014-05-05 21:59:19,2016-03-14 15:46:03
4,2362454995,Prize Rocket,PrizeFun52329,180,87,71,0,1,http://t.co/rPMDkKvchA,en-gb,...,NaN,NaN,NaN,NOTE TO WINNERS: A unique invitation code is n...,NaN,NaN,Wed Feb 26 09:57:57 +0000 2014,2014-02-26 10:57:57,2014-05-05 21:59:19,2016-03-14 15:46:03


In [101]:
traditional_spambots_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  100 non-null    int64  
 1   name                                100 non-null    object 
 2   screen_name                         100 non-null    object 
 3   statuses_count                      100 non-null    int64  
 4   followers_count                     100 non-null    int64  
 5   friends_count                       100 non-null    int64  
 6   favourites_count                    100 non-null    int64  
 7   listed_count                        100 non-null    int64  
 8   url                                 97 non-null     object 
 9   lang                                100 non-null    object 
 10  time_zone                           100 non-null    object 
 11  location                            1 non-null

In [102]:
traditional_spambots_3 = pd.read_csv("traditional_spambots_3_users.csv")
traditional_spambots_3.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,...,protected,verified,notifications,description,contributors_enabled,following,created_at,timestamp,crawled_at,updated
0,325403988,Borremans Bellman,borremanstpdri2,48,0,1,0,1,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,Tue Jun 28 07:15:18 +0000 2011,2011-06-28 09:15:18,2016-03-15 11:39:07,2016-03-15 11:39:07
1,3298943021,Reward Crazy,CrazyPrize66244,52,0,6,0,0,http://t.co/wCBL5xhOdq,en,...,NaN,NaN,NaN,Have a special code? Click the linky below:,NaN,NaN,Tue May 26 08:13:25 +0000 2015,2015-05-26 10:13:25,2016-03-15 11:39:07,2016-03-15 11:39:07
2,3305460917,Reward Patrol,CrazyPrize75229,39,0,4,0,0,http://t.co/c4Yxb1wFKj,en,...,NaN,NaN,NaN,Have an invitation password? Click the link be...,NaN,NaN,Mon Jun 01 09:58:30 +0000 2015,2015-06-01 11:58:30,2016-03-15 11:39:07,2016-03-15 11:39:07
3,179562837,Dana Shemesh,danashemesh,304,0,0,0,1,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,Tue Aug 17 16:09:13 +0000 2010,2010-08-17 18:09:13,2016-03-15 11:39:07,2016-03-15 11:39:07
4,179295032,Davina Vanwey,DavinaVanwey428,1883,0,0,0,1,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,Tue Aug 17 00:02:18 +0000 2010,2010-08-17 02:02:18,2016-03-15 11:39:07,2016-03-15 11:39:07


In [103]:
traditional_spambots_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  403 non-null    int64  
 1   name                                403 non-null    object 
 2   screen_name                         403 non-null    object 
 3   statuses_count                      403 non-null    int64  
 4   followers_count                     403 non-null    int64  
 5   friends_count                       403 non-null    int64  
 6   favourites_count                    403 non-null    int64  
 7   listed_count                        403 non-null    int64  
 8   url                                 336 non-null    object 
 9   lang                                403 non-null    object 
 10  time_zone                           100 non-null    object 
 11  location                            359 non-n

In [104]:
traditional_spambots_4 = pd.read_csv("traditional_spambots_4_users.csv")
traditional_spambots_4.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,...,protected,verified,notifications,description,contributors_enabled,following,created_at,timestamp,crawled_at,updated
0,21478911,TMJ- CLT Util Jobs,tmj_clt_util,4,344,295,0,5,https://t.co/DByWt45HZj,en,...,NaN,NaN,NaN,Follow this account for geo-targeted Utilities...,NaN,NaN,Sat Feb 21 12:04:47 +0000 2009,2009-02-21 13:04:47,2016-03-15 13:48:59,2016-03-15 13:48:59
1,21479094,TMJ - SFO Util Jobs,tmj_sfo_util,3,353,322,0,16,https://t.co/DByWt45HZj,en,...,NaN,NaN,NaN,Follow this account for geo-targeted Utilities...,NaN,NaN,Sat Feb 21 12:09:26 +0000 2009,2009-02-21 13:09:26,2016-03-15 13:48:59,2016-03-15 13:48:59
2,21479204,TMJ - WAS Util Jobs,tmj_dc_util,1,323,294,0,2,https://t.co/DByWt45HZj,en,...,NaN,NaN,NaN,Follow this account for geo-targeted Utilities...,NaN,NaN,Sat Feb 21 12:12:01 +0000 2009,2009-02-21 13:12:01,2016-03-15 13:48:59,2016-03-15 13:48:59
3,21479275,TMJ - JAX Util Jobs,tmj_jax_util,4,311,292,0,4,https://t.co/DByWt45HZj,en,...,NaN,NaN,NaN,Follow this account for geo-targeted Utilities...,NaN,NaN,Sat Feb 21 12:13:37 +0000 2009,2009-02-21 13:13:37,2016-03-15 13:48:59,2016-03-15 13:48:59
4,21479334,TMJ - CHI Util Jobs,tmj_chi_util,6,339,298,0,7,https://t.co/DByWt45HZj,en,...,NaN,NaN,NaN,Follow this account for geo-targeted Utilities...,NaN,NaN,Sat Feb 21 12:15:21 +0000 2009,2009-02-21 13:15:21,2016-03-15 13:48:59,2016-03-15 13:48:59


In [105]:
traditional_spambots_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  1128 non-null   int64  
 1   name                                1128 non-null   object 
 2   screen_name                         1128 non-null   object 
 3   statuses_count                      1128 non-null   int64  
 4   followers_count                     1128 non-null   int64  
 5   friends_count                       1128 non-null   int64  
 6   favourites_count                    1128 non-null   int64  
 7   listed_count                        1128 non-null   int64  
 8   url                                 1128 non-null   object 
 9   lang                                1128 non-null   object 
 10  time_zone                           1120 non-null   object 
 11  location                            1128 no

### Merging all datasets into combined dataset before exporting

In [106]:
# Add a new column to each dataframe to indicate bot or genuine
genuine_users['Type'] = 'Genuine'
social_spambots_1['Type'] = 'Bot'
social_spambots_2['Type'] = 'Bot'
social_spambots_3['Type'] = 'Bot'
traditional_spambots_1['Type'] = 'Bot'
traditional_spambots_2['Type'] = 'Bot'
traditional_spambots_3['Type'] = 'Bot'
traditional_spambots_4['Type'] = 'Bot'

# Concatenate the two samples into a single dataframe
combined_data = pd.concat([genuine_users, social_spambots_1, social_spambots_2, social_spambots_3,
                           traditional_spambots_1, traditional_spambots_2, traditional_spambots_3, traditional_spambots_4])

# Shuffle the combined dataframe
combined_data = combined_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the combined dataframe to a new CSV file
combined_data.to_csv("combined_data_full.csv", index=False)

In [ ]:
# Only use this step if you are loading the full dataset directly
# combined_data = pd.read_csv("combined_data_full.csv")

In [107]:
combined_data.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,...,description,contributors_enabled,following,created_at,timestamp,crawled_at,updated,test_set_1,test_set_2,Type
0,531256710,Christel Martillo,Martillodig,12515,787,1947,1,0,NaN,en,...,Lover of life - need I say more!,NaN,NaN,Tue Mar 20 12:07:58 +0000 2012,2012-03-20 13:07:58,2014-05-05 23:23:09,2016-03-15 15:39:59,NaN,1.0,Bot
1,72081097,TMJ-ON CstSrv Jobs,tmj_on_cstsrv,117,300,247,0,30,https://t.co/DByWt45HZj,en,...,Follow this account for geo-targeted Customer ...,NaN,NaN,Sun Sep 06 17:29:17 +0000 2009,2009-09-06 19:29:17,2016-03-15 13:49:10,2016-03-15 13:49:10,NaN,NaN,Bot
2,327060670,Federico Floria,Airolf,25930,10854,10606,418,66,http://t.co/0mQHlgxEmD,it,...,"Mika e' la mia vita, Supernatural la mia passi...",NaN,NaN,Thu Jun 30 22:59:43 +0000 2011,2011-07-01 00:59:43,2014-04-27 23:20:27,2016-03-15 14:13:49,1.0,NaN,Bot
3,2398910547,Southern Fields,southern_fields,730,1132,914,557,1,NaN,en,...,"Just a plain ol' country boy. I love God, Geor...",NaN,NaN,Mon Mar 10 07:48:09 +0000 2014,2014-03-10 08:48:09,2014-05-02 23:22:50,2016-03-15 14:16:12,1.0,NaN,Bot
4,69218476,Vanessa,Vanessa8w3v,1,153,568,0,0,http://xurl.jp/pc3a,NaN,...,NaN,NaN,NaN,1251353539000L,2009-08-27 08:12:19,2010-11-07 11:10:52,2016-03-14 17:08:43,NaN,NaN,Bot


In [108]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11017 entries, 0 to 11016
Data columns (total 43 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  11017 non-null  int64  
 1   name                                11016 non-null  object 
 2   screen_name                         11017 non-null  object 
 3   statuses_count                      11017 non-null  int64  
 4   followers_count                     11017 non-null  int64  
 5   friends_count                       11017 non-null  int64  
 6   favourites_count                    11017 non-null  int64  
 7   listed_count                        11017 non-null  int64  
 8   url                                 3508 non-null   object 
 9   lang                                10017 non-null  object 
 10  time_zone                           5015 non-null   object 
 11  location                            4908 

### Data preprocessing and encoding

In [109]:
# Drop columns that are unlikely to be useful or have too many missing values
columns_to_drop = [
    "id", "name", "screen_name", "url", "lang", "time_zone", "location",
    "default_profile", "default_profile_image", "geo_enabled", "profile_image_url",
    "profile_banner_url", "profile_use_background_image", "profile_background_image_url_https",
    "profile_text_color", "profile_image_url_https", "profile_sidebar_border_color",
    "profile_background_tile", "profile_sidebar_fill_color", "profile_background_image_url",
    "profile_background_color", "profile_link_color", "utc_offset", "is_translator",
    "follow_request_sent", "protected", "verified", "notifications", "contributors_enabled",
    "following", "created_at", "timestamp", "crawled_at", "updated", "test_set_1", "test_set_2"
]
combined_data_dropped = combined_data.drop(columns=columns_to_drop)
combined_data_dropped.head()

,statuses_count,followers_count,friends_count,favourites_count,listed_count,description,Type
0,12515,787,1947,1,0,Lover of life - need I say more!,Bot
1,117,300,247,0,30,Follow this account for geo-targeted Customer ...,Bot
2,25930,10854,10606,418,66,"Mika e' la mia vita, Supernatural la mia passi...",Bot
3,730,1132,914,557,1,"Just a plain ol' country boy. I love God, Geor...",Bot
4,1,153,568,0,0,NaN,Bot


In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Instantiate a TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), max_features=1000)

# Fit and transform the 'description' column
combined_data_dropped['description'] = combined_data_dropped['description'].fillna('')
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_data_dropped['description'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Concatenate original DataFrame with the TF-IDF DataFrame
combined_data_dropped = pd.concat([combined_data_dropped, tfidf_df], axis=1)

# Now drop the original 'description' column as it's been vectorized
combined_data_dropped.drop('description', axis=1, inplace=True)

# Encoding 'Type'
combined_data_dropped['Type'] = combined_data_dropped['Type'].map({'Bot': 1, 'Genuine': 0})

combined_data_dropped.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harrychang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,statuses_count,followers_count,friends_count,favourites_count,listed_count,Type,03,10,100,11,...,xx,ya,yeah,year,years,yo,york,young,youth,youtube
0,12515,787,1947,1,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,117,300,247,0,30,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,25930,10854,10606,418,66,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,730,1132,914,557,1,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,153,568,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Modelling with XGBoost

In [112]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix


# Assuming 'target' is your binary column indicating bot or not
X = combined_data_dropped.drop('Type', axis=1)
y = combined_data_dropped['Type']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the XGBoost classifier
model = XGBClassifier()
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
recall = recall_score(y_test, predictions)
precision = precision_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
conf_matrix = confusion_matrix(y_test, predictions)

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.984573502722323
Recall: 0.9856020942408377
Precision: 0.9920948616600791
F1 Score: 0.9888378200919239
Confusion Matrix:
[[ 664   12]
 [  22 1506]]


### Modelling with Random Forest

In [113]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)

# Fit the model
rf_model.fit(X_train, y_train)

# Make predictions
rf_predictions = rf_model.predict(X_test)

# Evaluate the model
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_recall = recall_score(y_test, rf_predictions)
rf_precision = precision_score(y_test, rf_predictions)
rf_f1 = f1_score(y_test, rf_predictions)
rf_conf_matrix = confusion_matrix(y_test, rf_predictions)

# Print the evaluation metrics
print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"Random Forest Recall: {rf_recall}")
print(f"Random Forest Precision: {rf_precision}")
print(f"Random Forest F1 Score: {rf_f1}")
print("Random Forest Confusion Matrix:")
print(rf_conf_matrix)

Random Forest Accuracy: 0.985934664246824
Random Forest Recall: 0.987565445026178
Random Forest Precision: 0.9921104536489151
Random Forest F1 Score: 0.9898327320432928
Random Forest Confusion Matrix:
[[ 664   12]
 [  19 1509]]


### Modelling with Logistic Regression

In [114]:
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression classifier
lr_model = LogisticRegression(max_iter=1000)

# Fit the model
lr_model.fit(X_train, y_train)

# Make predictions
lr_predictions = lr_model.predict(X_test)

# Evaluate the model
lr_accuracy = accuracy_score(y_test, lr_predictions)
lr_recall = recall_score(y_test, lr_predictions)
lr_precision = precision_score(y_test, lr_predictions)
lr_f1 = f1_score(y_test, lr_predictions)
lr_conf_matrix = confusion_matrix(y_test, lr_predictions)

# Print the evaluation metrics
print(f"Logistic Regression Accuracy: {lr_accuracy}")
print(f"Logistic Regression Recall: {lr_recall}")
print(f"Logistic Regression Precision: {lr_precision}")
print(f"Logistic Regression F1 Score: {lr_f1}")
print("Logistic Regression Confusion Matrix:")
print(lr_conf_matrix)

Logistic Regression Accuracy: 0.896551724137931
Logistic Regression Recall: 0.9882198952879581
Logistic Regression Precision: 0.877906976744186
Logistic Regression F1 Score: 0.9298029556650245
Logistic Regression Confusion Matrix:
[[ 466  210]
 [  18 1510]]
